## Imports: libraries and dataset

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import itertools
from statsmodels.tsa.stattools import adfuller

In [2]:
# Load the dataset
df = pd.read_csv('../../dataset/KAG_energydata_complete.csv')

# Convert the 'date' column to datetime format and set it as the index
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

## Arima model

In [18]:
# Focusing on the 'Appliances' energy consumption
series = df['Appliances']

In [19]:
# Perform ADF test to check for stationarity
result = adfuller(series)
print('ADF Statistic:', result[0])
print('p-value:', result[1])

ADF Statistic: -21.61637819803621
p-value: 0.0


In [20]:
# If the p-value is greater than 0.05, difference the series
if result[1] > 0.05:
    series = series.diff().dropna()

In [21]:
# Parameter tuning for ARIMA using a smaller range for efficiency
p = d = q = range(0, 3)
best_aic = float("inf")
best_params = None
num_combinations = len(list(itertools.product(p, d, q)))

print(f"Total combinations to evaluate: {num_combinations}")

Total combinations to evaluate: 27


In [ ]:
count = 0

for param in itertools.product(p, d, q):
    count += 1
    print(f"Evaluating combination {count}/{num_combinations}: ARIMA{param}")
    try:
        model = ARIMA(series, order=param)
        results = model.fit()
        if results.aic < best_aic:
            best_aic = results.aic
            best_params = param
    except Exception as e:
        print(f"Combination {param} failed with error: {e}")
        continue

print('Best ARIMA parameters:', best_params)
print('Best AIC:', best_aic)


In [23]:
# Fit the ARIMA model with the best parameters
model = ARIMA(series, order=best_params)
model_fit = model.fit()

/home/stas/.local/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 10min will be used.
  self._init_dates(dates, freq)
/home/stas/.local/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 10min will be used.
  self._init_dates(dates, freq)
/home/stas/.local/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 10min will be used.
  self._init_dates(dates, freq)


In [24]:
# Forecast
forecast_steps = 30  # Forecast for 30 time steps ahead
forecast = model_fit.forecast(steps=forecast_steps)

In [26]:
# Evaluate the model
y_train = series[:-forecast_steps]
y_test = series[-forecast_steps:]
y_pred = model_fit.predict(start=len(series)-forecast_steps, end=len(series)-1)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print('Mean Absolute Error (MAE):', mae)
print('Mean Squared Error (MSE):', mse)
print('Root Mean Squared Error (RMSE):', rmse)

Mean Absolute Error (MAE): 39.73663515185555
Mean Squared Error (MSE): 3713.644219823917
Root Mean Squared Error (RMSE): 60.93967689300557


## Sarima model

In [7]:
# Focusing on the 'Appliances' energy consumption and selecting exogenous variables
series = df['Appliances']
exog = df[['T1', 'RH_1', 'T2', 'RH_2', 'T_out', 'RH_out']]

In [25]:
# Fit the SARIMAX model with the selected parameters and exogenous variables
model = SARIMAX(series,
                order=(2, 0, 2),
                seasonal_order=(1, 0, 1, 24),
                exog=exog,
                enforce_stationarity=False,
                enforce_invertibility=False)
model_fit = model.fit(disp=False)

# Print summary of the model
print(model_fit.summary())

/home/stas/.local/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 10min will be used.
  self._init_dates(dates, freq)
/home/stas/.local/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency 10min will be used.
  self._init_dates(dates, freq)


In [21]:
# Forecast
forecast_steps = 30  # Forecast for 30 time steps ahead
exog_forecast = exog.iloc[-forecast_steps:].reset_index(drop=True)
forecast = model_fit.forecast(steps=forecast_steps, exog=exog_forecast)

In [24]:
# Evaluate the model
y_train = series[:-forecast_steps]
y_test = series[-forecast_steps:]
y_pred = model_fit.predict(start=len(series)-forecast_steps, end=len(series)-1, exog=exog_forecast)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print('Mean Absolute Error (MAE):', mae)
print('Mean Squared Error (MSE):', mse)
print('Root Mean Squared Error (RMSE):', rmse)

Mean Absolute Error (MAE): 43.669501705154616
Mean Squared Error (MSE): 3914.7614823868043
Root Mean Squared Error (RMSE): 62.56805480743991
